In [ ]:
# | default_exp logger

In [ ]:
# | export

from typing import *
import logging

In [ ]:
# | exporti

import logging.config
from contextlib import contextmanager, redirect_stdout, redirect_stderr
from io import StringIO
import traceback

In [ ]:
# | include: false

import pytest

In [ ]:
# | export

# Logger Levels
# CRITICAL = 50
# ERROR = 40
# WARNING = 30
# INFO = 20
# DEBUG = 10
# NOTSET = 0

should_supress_timestamps: bool = False


def supress_timestamps(flag: bool = True):
    """Supress logger timestamp

    Args:
        flag: If not set, then the default value **True** will be used to supress the timestamp
            from the logger messages
    """
    global should_supress_timestamps
    should_supress_timestamps = flag


def get_default_logger_configuration(level: int = logging.INFO) -> Dict[str, Any]:
    """Return the common configurations for the logger

    Args:
        level: Logger level to set

    Returns:
        A dict with default logger configuration

    """
    global should_supress_timestamps

    if should_supress_timestamps:
        FORMAT = "[%(levelname)s] %(name)s: %(message)s"
    else:
        FORMAT = "%(asctime)s.%(msecs)03d [%(levelname)s] %(name)s: %(message)s"

    DATE_FMT = "%y-%m-%d %H:%M:%S"

    LOGGING_CONFIG = {
        "version": 1,
        "disable_existing_loggers": False,
        "formatters": {
            "standard": {"format": FORMAT, "datefmt": DATE_FMT},
        },
        "handlers": {
            "default": {
                "level": level,
                "formatter": "standard",
                "class": "logging.StreamHandler",
                "stream": "ext://sys.stdout",  # Default is stderr
            },
        },
        "loggers": {
            "": {"handlers": ["default"], "level": level},  # root logger
        },
    }
    return LOGGING_CONFIG

Example on how to use `get_default_logger_configuration` function

In [ ]:
# collapse_output

get_default_logger_configuration()

{'version': 1,
 'disable_existing_loggers': False,
 'formatters': {'standard': {'format': '%(asctime)s.%(msecs)03d [%(levelname)s] %(name)s: %(message)s',
   'datefmt': '%y-%m-%d %H:%M:%S'}},
 'handlers': {'default': {'level': 20,
   'formatter': 'standard',
   'class': 'logging.StreamHandler',
   'stream': 'ext://sys.stdout'}},
 'loggers': {'': {'handlers': ['default'], 'level': 20}}}

In [ ]:
# | include: false

expected = {
    "version": 1,
    "disable_existing_loggers": False,
    "formatters": {
        "standard": {
            "format": "%(asctime)s.%(msecs)03d [%(levelname)s] %(name)s: %(message)s",
            "datefmt": "%y-%m-%d %H:%M:%S",
        }
    },
    "handlers": {
        "default": {
            "level": 20,
            "formatter": "standard",
            "class": "logging.StreamHandler",
            "stream": "ext://sys.stdout",
        }
    },
    "loggers": {"": {"handlers": ["default"], "level": 20}},
}
actual = get_default_logger_configuration()
assert actual == expected
actual

{'version': 1,
 'disable_existing_loggers': False,
 'formatters': {'standard': {'format': '%(asctime)s.%(msecs)03d [%(levelname)s] %(name)s: %(message)s',
   'datefmt': '%y-%m-%d %H:%M:%S'}},
 'handlers': {'default': {'level': 20,
   'formatter': 'standard',
   'class': 'logging.StreamHandler',
   'stream': 'ext://sys.stdout'}},
 'loggers': {'': {'handlers': ['default'], 'level': 20}}}

In [ ]:
# | export

logger_spaces_added: List[str] = []


def get_logger(
    name: str, *, level: int = logging.INFO, add_spaces: bool = True
) -> logging.Logger:
    """Return the logger class with default logging configuration.

    Args:
        name: Pass the __name__ variable as name while calling
        level: Used to configure logging, default value `logging.INFO` logs
            info messages and up.
        add_spaces:

    Returns:
        The logging.Logger class with default/custom logging configuration

    """
    config = get_default_logger_configuration(level=level)
    logging.config.dictConfig(config)

    logger = logging.getLogger(name)
    #     stack_size = len(traceback.extract_stack())
    #     def add_spaces_f(f):
    #         def f_with_spaces(msg, *args, **kwargs):
    #             cur_stack_size = len(traceback.extract_stack())
    #             msg = " "*(cur_stack_size-stack_size)*2 + msg
    #             return f(msg, *args, **kwargs)
    #         return f_with_spaces

    #     if name not in logger_spaces_added and add_spaces:
    #         logger.debug = add_spaces_f(logger.debug) # type: ignore
    #         logger.info = add_spaces_f(logger.info) # type: ignore
    #         logger.warning = add_spaces_f(logger.warning) # type: ignore
    #         logger.error = add_spaces_f(logger.error) # type: ignore
    #         logger.critical = add_spaces_f(logger.critical) # type: ignore
    #         logger.exception = add_spaces_f(logger.exception) # type: ignore

    #         logger_spaces_added.append(name)

    return logger

In [ ]:
# | include: false

assert type(get_logger(__name__)) == logging.Logger

with pytest.raises(TypeError) as e:
    get_logger()
assert "missing 1 required positional argument" in str(e.value)

In [ ]:
logger = get_logger(__name__)
logger.info("hello")
logger = get_logger(__name__)
logger.info("hello")


def f():
    logger.info("hello")


f()

22-11-28 07:01:10.892 [INFO] __main__: hello
22-11-28 07:01:10.893 [INFO] __main__: hello
22-11-28 07:01:10.894 [INFO] __main__: hello


Example on how to use `get_logger` function

In [ ]:
# collapse_output

logger = get_logger(__name__)

logger.debug("Debug")
logger.info("info")
logger.warning("Warning")
logger.error("Error")
logger.critical("Critical")

22-11-28 07:01:10.899 [INFO] __main__: info
22-11-28 07:01:10.899 [WARNING] __main__: Warning
22-11-28 07:01:10.900 [ERROR] __main__: Error
22-11-28 07:01:10.900 [CRITICAL] __main__: Critical


In [ ]:
# collapse_output

supress_timestamps()
logger = get_logger(__name__)

logger.debug("Debug")
logger.info("info")
logger.warning("Warning")
logger.error("Error")
logger.critical("Critical")

[INFO] __main__: info
[WARNING] __main__: Warning
[ERROR] __main__: Error
[CRITICAL] __main__: Critical


In [ ]:
# | export


def set_level(level: int):
    """Set logger level

    Args:
        level: Logger level to set
    """

    # Getting all loggers that has either airt or __main__ in the name
    loggers = [
        logging.getLogger(name)
        for name in logging.root.manager.loggerDict
        if ("airt" in name) or ("__main__" in name)
    ]

    for logger in loggers:
        logger.setLevel(level)

In [ ]:
level = logging.ERROR

set_level(level)

# Checking if the logger is set back to logging.WARNING in dev mode
print(logger.getEffectiveLevel())
assert logger.getEffectiveLevel() == level

logger.debug("This is a debug message")
logger.info("This is an info")
logger.warning("This is a warning")
logger.error("This is an error")

40
[ERROR] __main__: This is an error


In [ ]:
# Reset log level back to info
level = logging.INFO

set_level(level)
logger.info("something")

[INFO] __main__: something
